<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Topic_Modeling_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages

In [17]:
!pip install -U kaleido
!pip install bertopic
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [40]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import re
import string

import nltk
nltk.download('stopwords')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import kaleido

multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer:
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, doc):
    return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
# using spacy

nlp = spacy.load('en_core_web_lg')

class LemmaTokenizer:
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, doc):
    # return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

    return [self.wnl.t.lemma_ for t in nlp(doc)]

## Define path of files

In [22]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_8.2.2.xls',
 'Beamline_

In [24]:
files[-5]

'Beamline_8.3.1.xls'

## Helper Functions

In [25]:
def create_input(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']
  combined = list(df['Combined'])

  # remove elements with blank
  while " " in combined:
    combined.remove(" ")

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, ' ', l )
    comb_clean.append(mod_string)


  return comb_clean

In [36]:
lemma_spacy(["my names is Mudit","Hers name is Kritika"])

['-PRON- name be Mudit', 'her name be Kritika']

In [32]:
def lemma_spacy(df_combined):

  new_df= []
  for doc in df_combined:
    filt_combined = []
    for word in nlp(doc):
      filt_combined.append(word.lemma_)

    new_df.append(" ".join(filt_combined))

  return new_df

  

In [26]:
def remove_stop(df_combined):

  # remove stopwords

  new_df= []
  for doc in df_combined:
    filt_combined = []
    for word in word_tokenize(doc):

      if word.lower() not in stopwords.words('english'):
        # print(word)
        if word.lower() == "perovskites":
          filt_combined.append("perovskite")
        else:
          filt_combined.append(word)
    new_df.append(" ".join(filt_combined))

  return new_df

In [27]:
def berttopic_basic(df):

  topic_model = BERTopic()
  topics, probs = topic_model.fit_transform(df)

  # added lemmatization
  # update using lemmatization

  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model

In [28]:
import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

## example

In [35]:
df = pd.read_table(path + "Beamline_7.3.3.xls", on_bad_lines='skip')

# combine abstract and title
input_data = create_input(df)

# remove stopwords
filtered_input = remove_stop(input_data)

## Using BERTopic

In [14]:
start_time = time.time()
iter_version = "BERTopic_v2/"


for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input(df)

  # remove stopwords
  filtered_input = remove_stop(input_data)

  # run topic modeling algorithm
  model = berttopic_basic(filtered_input)

  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(model, file_path) 

  # visualize as html
  fig = model.visualize_barchart(top_n_topics = len(model.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))


File: 1 Beamline_1.4.xls


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


File: 1 done 
File: 2 Beamline_10.0.1.xls
File: 2 done 
File: 3 Beamline_10.3.1.xls
File: 3 done 
File: 4 Beamline_10.3.2.xls
File: 4 done 
File: 5 Beamline_11.0.1.xls
File: 5 done 
File: 6 Beamline_11.0.2.xls
File: 6 done 
File: 7 Beamline_11.3.1.xls
File: 7 done 
File: 8 Beamline_11.3.2.xls
File: 8 done 
File: 9 Beamline_12.0.1.1.xls
File: 9 done 
File: 10 Beamline_12.0.1.2.xls
File: 10 done 
File: 11 Beamline_12.0.1.4.xls
File: 11 done 
File: 12 Beamline_12.0.2.xls
File: 12 done 
File: 13 Beamline_12.2.1.xls
File: 13 done 
File: 14 Beamline_12.2.2.xls
File: 14 done 
File: 15 Beamline_12.3.1.xls
File: 15 done 
File: 16 Beamline_12.3.2.xls
File: 16 done 
File: 17 Beamline_2.1.xls
File: 17 done 
File: 18 Beamline_3.1.1.xls
File: 18 done 
File: 19 Beamline_3.3.2.xls
File: 19 done 
File: 20 Beamline_4.0.2.xls
File: 20 done 
File: 21 Beamline_4.0.3.1.xls
File: 21 done 
File: 22 Beamline_4.0.3.2.xls
File: 22 done 
File: 23 Beamline_4.2.2.xls
File: 23 done 
File: 24 Beamline_5.0.1.xls
File:

## Using SPECTER

In [45]:
def remove_punct(df):

  # remove punctuation
  text_col = [''.join(letter for letter in word if letter not in string.punctuation) for word in df]

  return text_col


In [30]:
def berttopic_specter(df):

  sentence_model = SentenceTransformer('allenai-specter')
  topic_model_specter = BERTopic(embedding_model=sentence_model)

  topics, probs = topic_model_specter.fit_transform(df)

  # added lemmatization
  # update using lemmatization

  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model_specter

In [46]:
start_time = time.time()
iter_version = "BERTopic_SPECTER_v1/"


for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input(df)

  # remove stopwords
  filtered_input = remove_stop(input_data)

  # remove stopwords
  filtered_input_2 = remove_punct(filtered_input)

  # run topic modeling algorithm
  model = berttopic_specter(filtered_input_2)

  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(model, file_path) 

  # visualize as html
  fig = model.visualize_barchart(top_n_topics = len(model.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))


File: 1 Beamline_1.4.xls
File: 1 done 
File: 2 Beamline_10.0.1.xls
File: 2 done 
File: 3 Beamline_10.3.1.xls
File: 3 done 
File: 4 Beamline_10.3.2.xls
File: 4 done 
File: 5 Beamline_11.0.1.xls
File: 5 done 
File: 6 Beamline_11.0.2.xls
File: 6 done 
File: 7 Beamline_11.3.1.xls
File: 7 done 
File: 8 Beamline_11.3.2.xls
File: 8 done 
File: 9 Beamline_12.0.1.1.xls
File: 9 done 
File: 10 Beamline_12.0.1.2.xls
File: 10 done 
File: 11 Beamline_12.0.1.4.xls
File: 11 done 
File: 12 Beamline_12.0.2.xls
File: 12 done 
File: 13 Beamline_12.2.1.xls
File: 13 done 
File: 14 Beamline_12.2.2.xls
File: 14 done 
File: 15 Beamline_12.3.1.xls
File: 15 done 
File: 16 Beamline_12.3.2.xls
File: 16 done 
File: 17 Beamline_2.1.xls
File: 17 done 
File: 18 Beamline_3.1.1.xls
File: 18 done 
File: 19 Beamline_3.3.2.xls
File: 19 done 
File: 20 Beamline_4.0.2.xls
File: 20 done 
File: 21 Beamline_4.0.3.1.xls


ValueError: ignored